根据大模型推理的结果，把数据上传到mongo数据库，再从mongo数据库查询数据导出最终的表格

## load prompt

In [2]:
import re
import pandas as pd
from tqdm import tqdm
from langchain.output_parsers import PydanticOutputParser

from prompts import industry_chain, PROMPT_TEMPLATE

In [3]:
from typing import List
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class IndustryClassifyResult(BaseModel):
    reason: str
    belong_to: List[str]
    evidence: List[str]
    confidence: float
    
parser = PydanticOutputParser(pydantic_object=IndustryClassifyResult)

In [4]:
import json
def load_jsonl(filename):
    with open(filename, "r") as f:
        data = []
        for line in f:
            data.append(json.loads(line))
        return data

## mongo client

In [5]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
# from dotenv import load_dotenv
import os

# 加载环境变量（建议将数据库连接信息放在 .env 文件中，避免硬编码）
# load_dotenv()  # 读取 .env 文件
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017/")  # 默认本地连接
DB_NAME = "industry"  # 数据库名
COLLECTION_NAME = "chain"  # 集合名（类似表）

In [8]:
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]
collection.create_index("corporation_name", unique=True)

'corporation_name_1'

## 上市公司

In [12]:
# df1 = pd.read_excel("data/煤化工_上市公司_经纬度_百度API.xlsx")

In [44]:
# data1 = load_jsonl("output/cisted_companies.jsonl")
data2 = load_jsonl("output/normal_companies.jsonl")

In [45]:
data2[0]

{'prompt': 'user\n你是一名产业研究分析师。请依据“企业经营范围”的**明示证据**，判断该企业在给定目标产业链中属于【上游 / 中游 / 下游】的哪一类（可多选）；若缺乏充分证据，请返回“均不属于”。**严禁臆测**。\n\n【判定准则】\n- **若未明确指向该产业链核心环节**，一律不作为归属依据。\n- 如经营范围同时覆盖多个环节，允许多选；但**仅限与目标产业链直接相关**的活动。\n\n【任务】\n1) 从“企业经营范围”中提取与目标产业链直接相关的**证据短语**（原文片段）。\n2) 基于证据，按上述准则判定归属。\n3) 若证据不足或只是笼统表述且未显式对接该产业链核心物料/设备/工艺/产品，返回“均不属于”。\n\n【输入】\n- 企业名: 陕西煤业化工集团有限责任公司\n- 企业经营范围: 煤炭开采、销售、加工和综合利用；煤化工产品、化学肥料和精细化工产品的研发、生产及销售；电力生产与供应；煤炭铁路运输（限自营铁路）；机械加工；煤矿专用设备、仪器及配件制造与修理；煤炭、化工、煤机的科研设计；煤田地质勘探；咨询服务；煤及伴生矿物深加工；矿山工程及工业和民用建筑；机电设备安装；矿井（建筑）工程设计；工程监理；建材销售；气体产品的制造和销售；火工、公路运输；物资仓储；高科技产业；农林业；自营代理各类商品及技术的进出口，但国家限定公司经营或禁止进出口的商品及技术除外。（其中煤炭开采、电力生产与供应、煤田地质勘察、气体产品制造、公路运输项目由集团公司所属企业凭许可证在有效期内经营）。（依法须经批准的项目，经相关部门批准后方可开展经营活动）\n- 目标产业链的描述文本: \n煤基聚酯产业链是以煤为原料，借助现代煤化工技术合成乙二醇、芳烃等中间体，进而延伸至聚酯（如PET）等高分子材料的产业体系。\n从产业链环节来看：\n上游：若煤化工企业涉及以煤为原料生产合成气（CO+H₂），或者利用合成气生产乙二醇（MEG）、甲醇等相关中间体，就属于上游环节。\n中游：企业主要进行聚酯（如PET）的生产，或者利用乙二醇等中间体生产二氯乙烷、乙二醇二甲醚、二乙二醇酯、二氧戊环等产品，属于中游环节。\n下游：企业从事聚酯纤维（像短纤、长丝用于纺纱，进而应用于服装、家纺等；工业长丝、民用长丝用于工业或民用领域）、聚酯瓶片、聚酯薄膜等终端产品的生产与应用，或者将

In [ ]:
print(data2[0]["prompt"])

user
你是一名产业研究分析师。请依据“企业经营范围”的**明示证据**，判断该企业在给定目标产业链中属于【上游 / 中游 / 下游】的哪一类（可多选）；若缺乏充分证据，请返回“均不属于”。**严禁臆测**。

【判定准则】
- **若未明确指向该产业链核心环节**，一律不作为归属依据。
- 如经营范围同时覆盖多个环节，允许多选；但**仅限与目标产业链直接相关**的活动。

【任务】
1) 从“企业经营范围”中提取与目标产业链直接相关的**证据短语**（原文片段）。
2) 基于证据，按上述准则判定归属。
3) 若证据不足或只是笼统表述且未显式对接该产业链核心物料/设备/工艺/产品，返回“均不属于”。

【输入】
- 企业名: 阳煤化工股份有限公司
- 企业经营范围: 一般项目：国内贸易代理【分支机构经营】；化肥销售【分支机构经营】；化工产品销售（不含许可类化工产品）【分支机构经营】；农副产品销售【分支机构经营】；金属材料销售【分支机构经营】；建筑材料销售【分支机构经营】；有色金属合金销售【分支机构经营】；机械零件、零部件销售【分支机构经营】；机械设备销售【分支机构经营】；电器辅件销售【分支机构经营】；仪器仪表销售【分支机构经营】；润滑油销售【分支机构经营】；以自有资金从事投资活动；技术服务、技术开发、技术咨询、技术交流、技术转让、技术推广；企业管理咨询；货物进出口【分支机构经营】；技术进出口【分支机构经营】；机械设备租赁。（除依法须经批准的项目外，凭营业执照依法自主开展经营活动）许可项目：危险化学品经营【分支机构经营】；危险废物经营【分支机构经营】。（依法须经批准的项目，经相关部门批准后方可开展经营活动，具体经营项目以相关部门批准文件或许可证件为准）
- 目标产业链的描述文本: 
煤基聚酯产业链是以煤为原料，借助现代煤化工技术合成乙二醇、芳烃等中间体，进而延伸至聚酯（如PET）等高分子材料的产业体系。
从产业链环节来看：
上游：若煤化工企业涉及以煤为原料生产合成气（CO+H₂），或者利用合成气生产乙二醇（MEG）、甲醇等相关中间体，就属于上游环节。
中游：企业主要进行聚酯（如PET）的生产，或者利用乙二醇等中间体生产二氯乙烷、乙二醇二甲醚、二乙二醇酯、二氧戊环等产品，属于中游环节。
下游：企业从事聚酯纤维（像短纤、长丝用于纺纱，进而应用于服装、家纺等；工业长丝、民

In [ ]:
# 上传数据到 mongo db
# belong_tos = []

err = 0
# for item in tqdm(data1):
for item in tqdm(data2):
    prompt = item["prompt"]
    corporation_name = re.search("- 企业名: (.*?)\n", prompt).group(1)
    chain_name = re.search("目标产业链的描述文本: \n(.*?)产业链", prompt).group(1)
    
    col_data = {
        f"{chain_name}_上游": 0,
        f"{chain_name}_中游": 0,
        f"{chain_name}_下游": 0,
    }

    try:
        d = parser.invoke(item["predict"])
        belong_to = d.belong_to
        # belong_tos.append(belong_to)
        for item in belong_to:
            if item == "上游":
                col_data[f"{chain_name}_上游"] = 1
            elif item == "中游":
                col_data[f"{chain_name}_中游"] = 1
            elif item == "下游":
                col_data[f"{chain_name}_下游"] = 1
    except:
        err += 1

    # MongoDB操作：更新或插入数据
    update_result = collection.update_one(
        {"corporation_name": corporation_name},
        {"$set": col_data},  # 使用$set避免覆盖原有字段
        upsert=True  # 如果不存在则插入新文档
    )

100%|██████████| 68831/68831 [00:45<00:00, 1509.23it/s]


## export2excel

In [47]:
# raw_df = pd.read_excel("data/煤化工_上市公司_经纬度_百度API.xlsx")
raw_df = pd.read_excel("data/煤化工_经纬度_百度API.xlsx")

In [48]:
raw_df.shape

(9833, 39)

In [49]:
add_data = []

for _, item in raw_df.iterrows():
    corporation_name = item["企业名称"]
    d = collection.find_one(
        {"corporation_name": corporation_name}, {"_id": 0}
    )
    add_data.append(d)

In [50]:
assert len(add_data) == raw_df.shape[0]

In [51]:
add_df = pd.DataFrame(add_data)

In [52]:
raw_df.shape

(9833, 39)

In [53]:
new_df = pd.concat([raw_df, add_df], axis=1)

In [54]:
# new_df.to_excel("output/煤化工_上市公司_经纬度_百度API_产业链.xlsx", index=False)
new_df.to_excel("output/煤化工_经纬度_百度API.xlsx", index=False)